<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jyp/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
class naive_bayes:
    def __init__(self,X_train,y_train):
        vertorizer=CountVectorizer()
        self.X_train_vec=vertorizer.fit_transform(X_train)
        self.word_ls=vertorizer.get_feature_names()
        self.y_train=y_train

    def score(self,k,doc):
        target_list=list(set(self.y_train))            
        matrix=[]
        for i in target_list:
            matrix.append(self.X_train_vec.toarray()[self.y_train==i].sum(axis=0))

        log_prior=np.log((np.array(matrix).T).sum(axis=0)/(np.array(matrix).T).sum())
        log_likelyhood=np.log(((np.array(matrix)).T+k)/((np.array(matrix).T).sum(axis=0)+2*k))
        probs=np.exp(log_likelyhood[[i for i,v in enumerate(self.word_ls) if v in doc.split()]].sum(axis=0)+log_prior)
        
        return probs/probs.sum()

    def predict(self,k,X_test):
        return np.array([self.score(k,doc).argmax() for doc in X_test])        

In [5]:
data=pd.DataFrame({"tokens":["me free lottery","free get free you","you free scholarship","free to contact me","you won award","you ticket lottery"],
    "spam":[1,1,0,0,0,1]})
data

,tokens,spam
0,me free lottery,1
1,free get free you,1
2,you free scholarship,0
3,free to contact me,0
4,you won award,0
5,you ticket lottery,1


In [6]:
X_train=data["tokens"]
y_train=data["spam"]

In [7]:
model=naive_bayes(X_train,y_train)
model.score(0.5,"free lottery")

array([0.125, 0.875])

In [8]:
model.score(0.5,"free dksldkjf")

array([0.41666667, 0.58333333])

In [9]:
model.score(0.5,"ghi we wefwe")

array([0.5, 0.5])

In [20]:
from sklearn.datasets import fetch_20newsgroups
twenty_train=fetch_20newsgroups(subset="train",shuffle=True)
twenty_test=fetch_20newsgroups(subset="test",shuffle=True)

In [21]:
X_train,y_train=twenty_train.data,twenty_train.target
X_test,y_test=twenty_test.data,twenty_test.target

In [ ]:
(naive_bayes(X_train,y_train).predict(0.5,X_test)==y_test).mean()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf=Pipeline([("vect",CountVectorizer()),
                   ("tfidf",TfidfTransformer()),
                   ("clf",MultinomialNB()),])
text_clf=text_clf.fit(twenty_train.data,twenty_train.target)
twenty_test=fetch_20newsgroups(subset="test",shuffle=True)
predicted=text_clf.predict(twenty_test.data)
np.mean(predicted==twenty_test.target)

0.7738980350504514